In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
# the hrf model to be used for data constructions
from nistats.hemodynamic_models import spm_hrf
# compute betaseries correlations
from nibetaseries.interfaces.nistats import BetaSeries
# make AR(1) correlated error terms
from statsmodels.tsa.arima_process import ArmaProcess
# generate optimal experimental designs
from neurodesign import optimisation,experiment
# make correlated betas
from scipy.linalg import cholesky
# numerical operations
import numpy as np
# convient to create tsvs
import pandas as pd
# create/operate on nifti images
import nibabel as nib

from multiprocessing import Pool
from collections import namedtuple, Counter
import tempfile
import os
import random

# Beta Series Simulations

Testing [NiBetaSeries](https://github.com/HBClab/NiBetaSeries) with 

In [ ]:
# repetition time of the MRI image
tr = 2
# number of experimental trials
n_trials = 80
# number of trial types (elijah wood and daniel radcliff)
n_trialtypes = 2
# minimum intertrial interval
iti_min = 3.5
# mean intertrial interval
iti_mean = 10
# maximum intertrial interval
iti_max = 20
# how to sample itis (mostly small, but a few larger ones)
iti_model = 'exponential'
# how long the stimulus is presented
stim_duration = 0.2
# stimulus probability (each stimulus is equally likely to occur)
stim_prob = [1 / n_trialtypes] * n_trialtypes
# contrasts of interest (dependent on n_trialtypes)
contrasts = [[1, 0], [0, 1], [1, -1]]
# resolution for the data generation process
des_res = 0.1
# autocorrelation for data
rho = 0.12
# set random seed for reproducibility
np.random.seed(12345)

In [ ]:
Experiment = experiment(
    TR=tr,
    n_trials=n_trials,
    P=stim_prob,
    C=contrasts,
    n_stimuli=n_trialtypes,
    rho=rho,
    resolution=des_res,
    stim_duration=stim_duration,
    ITImodel=iti_model,
    ITImin=iti_min,
    ITImean=iti_mean,
    ITImax=iti_max
    )

In [ ]:
# find best design
Designer = optimisation(
    experiment=Experiment,
    weights=[0, 0.25, 0.5, 0.25],
    preruncycles=2,
    cycles=100,
    optimisation='GA'
    )

Designer.optimise()

In [ ]:
# cannot use PopMax.bestdesign.Xnonconv because resolution information is lost
# (if I care about testing jitter)

In [ ]:
# https://quantcorner.wordpress.com/2018/02/09/generation-of-correlated-random-numbers-using-python/
# mean of the betas pulled from Mumford (2012) (hard coded!)
betas_mean = [5.0, 5.0, 5.0, 5.0]
# standard deviation for the betas (hard coded!)
betas_sd = [0.5, 0.5, 0.5, 0.5]
# beta network correlation for seeing elijah wood
wcorr_ew = 0.8
# beta network correlation for seeing daniel radcliff
wcorr_dr = 0.4
# the correlation between the elijah wood and daniel radcliff networks
bcorr = 0.0
# the number of trials per trial type
beta_matrix_rows = int(n_trials / n_trialtypes)
# the number of voxels to simulate
n_voxels = 2
# each trial type gets a column for each voxel
beta_matrix_columns = int(n_trialtypes * n_voxels)

# full correlation matrix (hard coded!)
corr_mat = np.array([[1.0, wcorr_ew, bcorr, bcorr],
                     [wcorr_ew, 1.0, bcorr, bcorr],
                     [bcorr, bcorr, 1.0, wcorr_dr],
                     [bcorr, bcorr, wcorr_dr, 1.0]])

# compute the (upper) Cholesky decomposition matrix
upper_chol = cholesky(corr_mat)

# generate random betas
rnd = np.random.normal(betas_mean, betas_sd,
                       size=(beta_matrix_rows, beta_matrix_columns))

# finally, compute the inner product of upper_chol and rnd
betas = rnd @ upper_chol

# see how closely generated data matches assumptions
ground_truth = np.corrcoef(betas.T)

# elijah wood's ground truth beta correlation (hard coded!)
truth_ew = ground_truth[0, 1]
# daniel radcliff's ground truth beta correlation (hard coded!)
truth_dr = ground_truth[2, 3]
print('Elijah Wood within network correlation:     {}'.format(truth_ew))
print('Daniel Radcliff within network correlation: {}'.format(truth_dr))

In [ ]:
cond_order = Designer.bestdesign.order
# divide by design resolution to have same resolution as experiment generation process
onsets = Designer.bestdesign.onsets / des_res
onsets = onsets.astype(int)
# allocate design matrix (one column per trial)
X = np.zeros((int(Designer.bestdesign.experiment.duration / des_res), onsets.shape[0]))
# allocate betas (two columns for 2 voxels)
B = np.zeros((onsets.shape[0], 2))
# the stimulus duration represented in the design resolution
stim_duration_msec = int(stim_duration / des_res)
# oversampling at the rate of the design resolution
sampling_rate = int(tr / des_res)
# counters for elijah wood and daniel radcliff stimuli
cond_ew = 0
cond_dr = 0

# create the design matrix for the data generation process
for idx, (cond, onset) in enumerate(zip(cond_order, onsets)):
    # set the design matrix
    X[onset:onset+stim_duration_msec, idx] = 1
    X[:, idx] = np.convolve(X[:, idx], spm_hrf(tr, oversampling=sampling_rate))[0:X.shape[0]]
    # set the beta for the trial depending on condition
    if cond == 0:
        B[idx, :] = betas[cond_ew, 0:2]
        cond_ew += 1
    elif cond == 1:
        B[idx, :] = betas[cond_dr, 2:4]
        cond_dr += 1

# downsample X so it's back to TR resolution
X = X[::sampling_rate, :]

In [ ]:
n_trs = int(Designer.bestdesign.experiment.duration / tr)

In [ ]:
# AR(1) process with 1 lag with a .12 correlation (Mumford et al. 2012)
sd_err = 0.08
ar = np.array([1, -rho])
ap = ArmaProcess(ar)
err = ap.generate_sample((n_trs, 2), scale=sd_err, axis=0)

In [ ]:
Y = X @ B + err

In [ ]:
# define signal to noise: http://www.scholarpedia.org/article/Signal-to-noise_ratio
signal = X @ B
noise = err

np.mean(signal[:,0] ** 2) / np.var(err)

In [ ]:
plt.plot(Y)

In [ ]:
# Make events file
events_file = 'events.tsv'
collector = {'onset': [], 
             'duration': [], 
             'correct': [],
             'trial_type': []}
for cond, onset in zip(cond_order, onsets):
    if cond == 0:
        collector['trial_type'].append('elijah_wood')
    elif cond == 1:
        collector['trial_type'].append('daniel_radcliffe')

    collector['onset'].append(onset * des_res)
    collector['duration'].append(stim_duration)
    collector['correct'].append(1)

events_df = pd.DataFrame.from_dict(collector)
events_df.to_csv(events_file, sep='\t', index=False)

In [ ]:
# create bold nifti
bold_file = 'bold_file.nii.gz'
bold_data = np.array([[Y.T]])
bold_img = nib.Nifti1Image(bold_data, np.eye(4))
bold_img.to_filename(bold_file)

# create mask nifti
mask_file = 'brainmask.nii.gz'
mask_data = np.array([[[1, 1]]], dtype=np.int16)
mask_img = nib.Nifti1Image(mask_data, np.eye(4))
mask_img.to_filename(mask_file)

In [ ]:
# bold_metadata
bold_metadata = {"RepetitionTime": tr, "TaskName": "whodis"}

In [ ]:
beta_series = BetaSeries(bold_file=bold_file,
                         bold_metadata=bold_metadata,
                         confounds_file=None,
                         events_file=events_file,
                         hrf_model='spm',
                         low_pass=None,
                         mask_file=mask_file,
                         selected_confounds=None,
                         smoothing_kernel=None)

In [ ]:
result = beta_series.run()

In [ ]:
result.outputs.beta_maps

In [ ]:
for bmap in result.outputs.beta_maps:
    if 'elijah_wood' in bmap:
        bm_ew = nib.load(bmap)
    elif 'daniel_radcliffe' in bmap:
        bm_dr = nib.load(bmap)

In [ ]:
betas_ew = np.squeeze(bm_ew.get_data())
betas_dr = np.squeeze(bm_dr.get_data())

In [ ]:
np.corrcoef(betas_dr)

In [ ]:
truth_ew - np.corrcoef(betas_ew)[0, 1]

# Simulations

I would like to test the impact of jitter on correlations, the length of the ITI, how much noise is necessary/how to represent noise

In [ ]:
import copyreg
import types

def _pickle_method(method):
    """
    Author: Steven Bethard 
    http://bytes.com/topic/python/answers/552476-why-cant-you-pickle-instancemethods
    """
    func_name = method.im_func.__name__
    obj = method.im_self
    cls = method.im_class
    cls_name = ''
    if func_name.startswith('__') and not func_name.endswith('__'):
        cls_name = cls.__name__.lstrip('_')
    if cls_name:
        func_name = '_' + cls_name + func_name
    return _unpickle_method, (func_name, obj, cls)


def _unpickle_method(func_name, obj, cls):
    """
    Author: Steven Bethard
    http://bytes.com/topic/python/answers/552476-why-cant-you-pickle-instancemethods
    """
    for cls in cls.mro():
        try:
            func = cls.__dict__[func_name]
        except KeyError:
            pass
        else:
            break
    return func.__get__(obj, cls)

# This call to copy_reg.pickle allows you to pass methods as the first arg to
# mp.Pool methods. If you comment out this line, `pool.map(self.foo, ...)` results in
# PicklingError: Can't pickle <type 'instancemethod'>: attribute lookup
# __builtin__.instancemethod failed

copyreg.pickle(types.MethodType, _pickle_method, _unpickle_method)

In [16]:
class BetaSeriesSimulation:


    def __init__(self, tr=2, n_trials=80, n_trialtypes=2,
                 iti_min=2, iti_mean=4, iti_max=16,
                 iti_model='exponential', stim_duration=0.2,
                 contrasts=[[1, 0], [0, 1], [1, -1]],
                 des_res=0.1, rho=0.12, sd_err=0.08,
                 wcorr_ew=0.0, wcorr_dr=0.8, bcorr=0.,
                 n_simulations=500, n_proc=1):
        """Class for performing and containing results from simulations

        Parameters
        ----------

        tr : int
            repetition time of the fMRI bold series
        n_trials : int
            number of experimental trials
        n_trialtypes : int
            number of trial types
        iti_min : float
            minimum intertrial interval
        iti_mean : float
            mean intertrial interval
        iti_max : float
            maximum intertrial interval
        iti_model : str
            distribution to sample iti's from 
            (choices: “fixed”,”uniform”,”exponential”)
        stim_duration : float
            how long each stimulus is presented
        contrasts : list
            contrasts of interest (dependent on n_trialtypes)
        des_res : float
            design resolution for the data generation process
        rho : float
            AR(1) correlation coefficient
        sd_err : float
            the standard deviation for the error term
        n_simulations : int
            number of iterations for the simulation


        Attributes
        ----------

        tr : int
            repetition time of the fMRI bold series
        n_trials : int
            number of experimental trials
        n_trialtypes : int
            number of trial types
        iti_min : float
            minimum intertrial interval
        iti_mean : float
            mean intertrial interval
        iti_max : float
            maximum intertrial interval
        iti_model : str
            distribution to sample iti's from 
            (choices: “fixed”,”uniform”,”exponential”)
        stim_duration : float
            how long each stimulus is presented
        contrasts : list
            contrasts of interest (dependent on n_trialtypes)
        des_res : float
            design resolution for the data generation process
        rho : float
            AR(1) correlation coefficient
        sd_err : float
            the standard deviation for the error term
        n_simulations : int
            number of iterations for the simulation
        Designer : neurodesign.optimisation
            optimized experimental design object
        simulation_results : pandas.DataFrame
            the collection of the true correlations and the 
            estimated correlations from betaseries correlations.
        """

        self.tr = tr
        self.n_trials = n_trials
        self.n_trialtypes = n_trialtypes
        self.iti_min = iti_min
        self.iti_mean = iti_mean
        self.iti_max = iti_max
        self.iti_model = iti_model
        self.stim_duration = stim_duration
        self.contrasts = contrasts
        self.des_res = des_res
        self.rho = rho
        self.sd_err = sd_err
        self.tmp_dir = tempfile.mkdtemp(prefix='simulation_')
        self.n_simulations = n_simulations
        self.wcorr_ew = wcorr_ew
        self.wcorr_dr = wcorr_dr
        self.bcorr = bcorr
        self.n_proc = n_proc
        
        # set by _make_design
        self.Designer = None

        # set by run_simulations
        self.simulation_results = None

        
    def make_design(self):
        """generates an optimized experimental design
        """
        # stimulus probability (each stimulus is equally likely to occur)
        stim_prob = [1 / self.n_trialtypes] * self.n_trialtypes
        
        # setup experimental parameters
        Experiment = experiment(
            TR=self.tr,
            n_trials=self.n_trials,
            P=stim_prob,
            C=self.contrasts,
            n_stimuli=self.n_trialtypes,
            rho=self.rho,
            resolution=self.des_res,
            stim_duration=self.stim_duration,
            ITImodel=self.iti_model,
            ITImin=self.iti_min,
            ITImean=self.iti_mean,
            ITImax=self.iti_max)

        # find best design
        Designer = optimisation(
            experiment=Experiment,
            weights=[0, 0.25, 0.5, 0.25],
            preruncycles=2,
            cycles=100,
            optimisation='GA')

        # keep optimizing until there are an equal...
        # ..number of trials for each trialtype
        optimise = True
        while optimise:
            Designer.optimise()
            trial_count = list(Counter(Designer.bestdesign.order).values())
            # try again if conditions do have equal trials
            optimise = not all(x == trial_count[0] for x in trial_count)
        
        self.Designer = Designer
    

    def run_simulations(self):
        """simulates data and performs correlations
        """
        import pdb; pdb.set_trace()

        with Pool(processes=self.n_proc) as pool:
            sim_res = pool.map(self._run_sim, range(self.n_simulations))
        
        sim_res_dict = {
            k: [d.get(k) for d in sim_res]
            for k in set().union(*sim_res)}
        # make an analyzable dataframe from the simulated results
        self.simulation_results = pd.DataFrame.from_dict(sim_res_dict)

    def _run_sim(self, num):
        cond_order = self.Designer.bestdesign.order
        onsets = self.Designer.bestdesign.onsets
        duration = self.Designer.bestdesign.experiment.duration
        # set randomization (otherwise processes spawned at the same time have the same result)
        np.random.seed(num)
        random.seed(num)
        # number of simulation
        simulation_results_dict = {'num': num}
        # simulate data
        gen_betas = self._generate_betas()
        simulation_results_dict['true_corr_ew'] = gen_betas.true_corr_ew
        simulation_results_dict['true_corr_dr'] = gen_betas.true_corr_dr

        sim_data = self._simulate_data(gen_betas.betas, cond_order, onsets, duration)
        simulation_results_dict['snr'] = sim_data.snr

        events_file = self._make_events_tsv(cond_order, onsets, self.stim_duration)
        bold_file = self._make_bold_nifti(Y=sim_data.Y)
        mask_file = self._make_mask_nifti()
        bold_metadata = {"RepetitionTime": self.tr, "TaskName": "whodis"}

        beta_results = self._run_betaseries(bold_file, bold_metadata, events_file, mask_file)
        simulation_results_dict['corr_ew'] = beta_results.corr_ew 
        simulation_results_dict['corr_dr'] = beta_results.corr_dr
        
        return simulation_results_dict


    def _generate_betas(self):
        """
        makes the simulated beta values
        
        Returns
        -------
        
        betas : numpy.array
            numpy array size (n_trials / n_trialtypes) x (n_trials * n_voxels)
            to give each trialtype their unique betas per voxel
        
        true_corr_ew : float
            the correlation between the two elijah wood voxels
        
        true_corr_dr : float
            the correlation between the two daniel radcliffe voxels
            
        """
        # https://quantcorner.wordpress.com/2018/02/09/generation-of-correlated-random-numbers-using-python/
        # mean of the betas pulled from Mumford (2012) (hard coded!)
        betas_mean = [5.0, 5.0, 5.0, 5.0]
        # standard deviation for the betas (hard coded!)
        betas_sd = [0.5, 0.5, 0.5, 0.5]
        # beta network correlation for seeing elijah wood
        # beta network correlation for seeing daniel radcliff
        # the correlation between the elijah wood and daniel radcliff networks
        # the number of trials per trial type
        beta_matrix_rows = int(self.n_trials / self.n_trialtypes)
        # the number of voxels to simulate
        n_voxels = 2
        # each trial type gets a column for each voxel
        beta_matrix_columns = int(self.n_trialtypes * n_voxels)

        # full correlation matrix (hard coded!)
        corr_mat = np.array([[1.0, self.wcorr_ew, self.bcorr, self.bcorr],
                             [self.wcorr_ew, 1.0, self.bcorr, self.bcorr],
                             [self.bcorr, self.bcorr, 1.0, self.wcorr_dr],
                             [self.bcorr, self.bcorr, self.wcorr_dr, 1.0]])

        # compute the (upper) Cholesky decomposition matrix
        upper_chol = cholesky(corr_mat)

        # generate random betas
        rnd = np.random.normal(betas_mean, betas_sd,
                               size=(beta_matrix_rows, beta_matrix_columns))

        # finally, compute the inner product of upper_chol and rnd
        betas = rnd @ upper_chol

        # see how closely generated data matches assumptions
        ground_truth = np.corrcoef(betas.T)

        # elijah wood's ground truth beta correlation (hard coded!)
        true_corr_ew = ground_truth[0, 1]
        # daniel radcliff's ground truth beta correlation (hard coded!)
        true_corr_dr = ground_truth[2, 3]

        SimulatedBetas = namedtuple('SimulatedBetas', 'betas true_corr_ew true_corr_dr')

        return SimulatedBetas(betas=betas, true_corr_ew=true_corr_ew, true_corr_dr=true_corr_dr)

    
    def _simulate_data(self, betas, cond_order, onsets, duration):
        """simulates the data for the voxels
        
        Parameters
        ----------
        
        betas : numpy.array
            numpy array size (n_trials / n_trialtypes) x (n_trials * n_voxels)
            to give each trialtype their unique betas per voxel
        cond_order : list
            each entry is an integer representing the trialtype for that
            particular trial
        onsets : numpy.array
            identifies each onset (in seconds) for a trial to occur
        duration : float
            the total length (in seconds) of the experiment
        
        Returns
        -------
        
        Y : numpy.array
            the simulated data with the size n_volumes x n_voxels
        snr : float
            a measure of signal to noise
        """
        # divide by design resolution to have same resolution as experiment generation process
        onsets = onsets / self.des_res
        onsets = onsets.astype(int)
        # allocate design matrix (one column per trial)
        X = np.zeros((int(duration / self.des_res), onsets.shape[0]))
        # allocate betas (two columns for 2 voxels)
        B = np.zeros((onsets.shape[0], 2))
        # the stimulus duration represented in the design resolution
        stim_duration_msec = int(self.stim_duration / self.des_res)
        # oversampling at the rate of the design resolution
        sampling_rate = int(self.tr / self.des_res)
        # counters for elijah wood and daniel radcliff stimuli
        cond_ew = 0
        cond_dr = 0

        # create the design matrix for the data generation process
        for idx, (cond, onset) in enumerate(zip(cond_order, onsets)):
            # set the design matrix
            X[onset:onset+stim_duration_msec, idx] = 1
            X[:, idx] = np.convolve(X[:, idx], spm_hrf(self.tr, oversampling=sampling_rate))[0:X.shape[0]]
            # set the beta for the trial depending on condition
            if cond == 0:
                B[idx, :] = betas[cond_ew, 0:2]
                cond_ew += 1
            elif cond == 1:
                B[idx, :] = betas[cond_dr, 2:4]
                cond_dr += 1

        # downsample X so it's back to TR resolution
        X = X[::sampling_rate, :]

        # make the noise component
        n_trs = int(duration / self.tr)
        ar = np.array([1, -self.rho]) # statmodels says to invert rho
        ap = ArmaProcess(ar)
        n_voxels = 2
        err = ap.generate_sample((n_trs, n_voxels), scale=self.sd_err, axis=0)

        # define signal to noise: http://www.scholarpedia.org/article/Signal-to-noise_ratio
        signal = X @ B
        noise = err

        snr = signal.var() / err.var()

        # simulated data
        Y = signal + noise

        SimulatedData = namedtuple('SimulatedData', 'Y snr')

        return SimulatedData(Y=Y, snr=snr)


    def _make_events_tsv(self, cond_order, onsets, duration):
        """creates events.tsv file
        
        Parameters
        ----------
        
        cond_order : list
            each entry is an integer representing the trialtype for that
            particular trial
        onsets : numpy.array
            identifies each onset (in seconds) for a trial to occur
        stim_duration : float
            the stimulus duration
            
        Returns
        -------
        
        events_file : str
            pathname to the events file
        """
        events_file = os.path.join(self.tmp_dir, 'events.tsv')
        collector = {'onset': [], 
                     'duration': [], 
                     'correct': [],
                     'trial_type': []}
        for cond, onset in zip(cond_order, onsets):
            if cond == 0:
                collector['trial_type'].append('elijah_wood')
            elif cond == 1:
                collector['trial_type'].append('daniel_radcliffe')

            collector['onset'].append(onset)
            collector['duration'].append(duration)
            collector['correct'].append(1)

        events_df = pd.DataFrame.from_dict(collector)
        events_df.to_csv(events_file, sep='\t', index=False)
        
        return events_file


    def _make_bold_nifti(self, Y):
        """creates bold file
        
        Paramters
        ---------
        
        Y : numpy.array
            the simulated data with the size n_volumes x n_voxels
            
        Returns
        -------
        
        bold_file : str
            pathname to the bold file
        """
        bold_file = os.path.join(self.tmp_dir, 'bold_file.nii.gz')
        bold_data = np.array([[Y.T]])
        bold_img = nib.Nifti1Image(bold_data, np.eye(4))
        bold_img.to_filename(bold_file)

        return bold_file
    

    def _make_mask_nifti(self):
        """creates mask file (assumes 2 voxels)
            
        Returns
        -------
        
        mask_file : str
            pathname to the mask file
        """
        mask_file = os.path.join(self.tmp_dir, 'brainmask.nii.gz')
        mask_data = np.array([[[1, 1]]], dtype=np.int16)
        mask_img = nib.Nifti1Image(mask_data, np.eye(4))
        mask_img.to_filename(mask_file)

        return mask_file


    def _run_betaseries(self, bold_file, bold_metadata, events_file, mask_file):
        """runs betaseries correlations
        
        Parameters
        ----------
        
        bold_file : str
            pathname to a bold file
        bold_metadata : dict
            dictionary containing tr and task information
        events_file : str
            pathname to an events file
        mask_file : str
            pathname to a mask file
            
        Returns
        -------
        
        corr_ew : float
            estimated betaseries correlation for elijah wood
        corr_dr : float
            estimated betaseries correlation for daniel radcliffe
        """
        beta_series = BetaSeries(bold_file=bold_file,
                                 bold_metadata=bold_metadata,
                                 confounds_file=None,
                                 events_file=events_file,
                                 hrf_model='spm',
                                 low_pass=None,
                                 mask_file=mask_file,
                                 selected_confounds=None,
                                 smoothing_kernel=None)

        result = beta_series.run(cwd=self.tmp_dir)

        for bmap in result.outputs.beta_maps:
            if 'elijah_wood' in bmap:
                bm_ew = nib.load(bmap)
            elif 'daniel_radcliffe' in bmap:
                bm_dr = nib.load(bmap)

        betas_ew = np.squeeze(bm_ew.get_data())
        betas_dr = np.squeeze(bm_dr.get_data())
        corr_ew = np.corrcoef(betas_ew)[0, 1]
        corr_dr = np.corrcoef(betas_dr)[0, 1]

        ModeledCorrs = namedtuple('ModeledCorrs', ' corr_ew corr_dr beta_res')
        return ModeledCorrs(corr_ew=corr_ew, corr_dr=corr_dr, beta_res=result)

In [17]:
sim_low_err = BetaSeriesSimulation(sd_err=0.001, n_trials=60, n_simulations=10, n_proc=4)
# sim_med_err = BetaSeriesSimulation(sd_err=0.01, n_trials=60, n_simulations=10, n_proc=4)
# sim_high_err = BetaSeriesSimulation(sd_err=0.1, n_trials=60, n_simulations=10, n_proc=4)

In [18]:
sim_low_err.make_design()
# sim_med_err.make_design()
# sim_high_err.make_design()

/home/james/.conda/envs/sim_test/lib/python3.6/site-packages/neurodesign/classes.py:410: UserWarning: Warning: the resolution is adjusted to be a multiple of the TR.  New resolution: 0.100000
  warnings.warn("Warning: the resolution is adjusted to be a multiple of the TR.  New resolution: %f"%self.resolution)
100% |########################################################################|
100% |########################################################################|


In [19]:
sim_low_err.run_simulations()
# sim_med_err.run_simulations()/
# sim_high_err.run_simulations()

> <ipython-input-16-cc468b8b53e2>(154)run_simulations()
-> with Pool(processes=self.n_proc) as pool:


(Pdb)  c


/home/james/.conda/envs/sim_test/lib/python3.6/site-packages/nistats/hemodynamic_models.py:55: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  time_stamps = np.linspace(0, time_length, float(time_length) / dt)
/home/james/.conda/envs/sim_test/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:221: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
Computing run 1 out of 1 runs (go take a coffee, a big one)
/home/james/.conda/envs/sim_test/lib/python3.6/site-packages/nistats/hemodynamic_models.py:268: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  frame_times.max() * (1 + 1. / (n - 1)), n_hr)
/home/james/.conda/envs/sim_test/lib/python3.6/site-packages/nistats/hemodynamic_models.py:55: Dep

In [ ]:
sim_low_err.simulation_results.to_csv('low_err_summary.tsv', sep='\t', index=False)
sim_med_err.simulation_results.to_csv('med_err_summary.tsv', sep='\t', index=False)
sim_high_err.simulation_results.to_csv('high_err_summary.tsv', sep='\t', index=False)

In [20]:
sim_low_err.simulation_results

,true_corr_dr,num,corr_dr,true_corr_ew,corr_ew,snr
0,0.752497,0,0.631619,-0.061246,0.373458,81759.135191
1,0.703701,1,0.563763,-0.405438,0.318074,74314.296148
2,0.616821,2,0.442803,0.149545,0.351542,65053.263832
3,0.618401,3,0.434904,0.288569,0.290575,64789.865747
4,0.857474,4,0.761790,-0.100787,0.192573,76386.068413
5,0.870523,5,0.848375,0.127690,0.321357,74603.779227
6,0.791251,6,0.802874,-0.052369,0.269050,80293.805299
7,0.710752,7,0.821401,0.166749,0.185241,80905.039506
8,0.873389,8,0.622167,-0.061017,0.000073,72847.954623
9,0.888178,9,0.766720,0.153436,0.037543,74012.617293


In [ ]:
sim_med_err.simulation_results

In [ ]:
sim_high_err.simulation_results

In [ ]:
cond_order = sim_low_err.Designer.bestdesign.order
onsets = sim_low_err.Designer.bestdesign.onsets
duration = sim_low_err.Designer.bestdesign.experiment.duration

In [ ]:
gen_betas = sim_low_err._generate_betas()

In [ ]:
gen_betas.betas

In [ ]:
sim_data = sim_low_err._simulate_data(gen_betas.betas, cond_order, onsets, duration)
sim_data

In [ ]:
plt.plot(sim_data.Y)

In [ ]:
events_file = sim_low_err._make_events_tsv(cond_order, onsets, duration)
bold_file = sim_low_err._make_bold_nifti(Y=sim_data.Y)
mask_file = sim_low_err._make_mask_nifti()
bold_metadata = {"RepetitionTime": sim_low_err.tr, "TaskName": "whodis"}

beta_results = sim_low_err._run_betaseries(bold_file, bold_metadata, events_file, mask_file)

In [ ]:

for bmap in beta_results.beta_res.outputs.beta_maps:
    if 'elijah_wood' in bmap:
        bm_ew = nib.load(bmap)
    elif 'daniel_radcliffe' in bmap:
        bm_dr = nib.load(bmap)

In [ ]:
bm_dr.get_data()

In [ ]:
gen_betas.betas

In [ ]:
sim_low_err = BetaSeriesSimulation(sd_err=0.001, n_trials=60, n_simulations=10)

In [ ]:
for x in range(0,10):
    cont=False
    for i in range(0,10):
        print("x: ", str(x), " i: ", str(i))
        if x == i:
            cont=True
            break
    if cont:
        continue
    # this code should never run
    print("new")

In [ ]:
from multiprocessing import Pool, TimeoutError
import time
import os

def f(x):
    return {x: x}

if __name__ == '__main__':
    # start 4 worker processes
    with Pool(processes=4) as pool:

        # print "[0, 1, 4,..., 81]"
        print(pool.map(f, range(10)))

        # print same numbers in arbitrary order
        for i in pool.imap_unordered(f, range(10)):
            print(i)

        # evaluate "f(20)" asynchronously
        res = pool.apply_async(f, (20,))      # runs in *only* one process
        print(res.get(timeout=1))             # prints "400"

        # evaluate "os.getpid()" asynchronously
        res = pool.apply_async(os.getpid, ()) # runs in *only* one process
        print(res.get(timeout=1))             # prints the PID of that process

        # launching multiple evaluations asynchronously *may* use more processes
        multiple_results = [pool.apply_async(os.getpid, ()) for i in range(4)]
        print([res.get(timeout=1) for res in multiple_results])

        # make a single worker sleep for 10 secs
        res = pool.apply_async(time.sleep, (10,))
        try:
            print(res.get(timeout=1))
        except TimeoutError:
            print("We lacked patience and got a multiprocessing.TimeoutError")

        print("For the moment, the pool remains available for more work")

    # exiting the 'with'-block has stopped the pool
    print("Now the pool is closed and no longer available")

In [ ]:
import copy_reg

In [22]:
rt = np.random.normal(0, 1, size=200)

In [25]:
rt.var()

0.9546526019990398